In [ ]:
import scan_csv
from FastSlidingWindow import *
from Util import *
from bbd100k_loader import *
from scan_csv import progress

loader = BBD100K_Loader(False)
color_map = generate_color_from_categories(loader.category_dict)

In [ ]:
loader.image_count

In [1]:
import tensorflow as tf
import numpy as np
tf.enable_eager_execution()
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
tf.Session(config=config).close()

In [15]:
def calc_iou(Y_pred, Y):
    #print(Y_pred.shape)
    zeros = np.zeros([Y_pred.shape[0], Y_pred.shape[1], Y_pred.shape[2], 1], dtype=np.float32)
    y2 = tf.reshape(Y_pred[:, :, :, 2], [Y_pred.shape[0], Y_pred.shape[1], Y_pred.shape[2], 1])
    y4 = tf.reshape(Y_pred[:, :, :, 4], [Y_pred.shape[0], Y_pred.shape[1], Y_pred.shape[2], 1])
    pred_x_w = tf.where(y2 > 0.0, y2, zeros) 
    pred_x_h = tf.where(y4 > 0.0, y4, zeros) 
    
    x1_t = Y_pred[:, :, :, 1:2] - Y[:, :, :, 2:3] / 2.0
    x2_t = Y_pred[:, :, :, 1:2] + Y[:, :, :, 2:3] / 2.0
    
    y1_t = Y_pred[:, :, :, 3:4] - Y[:, :, :, 4:5] / 2.0
    y2_t = Y_pred[:, :, :, 3:4] + Y[:, :, :, 4:5] / 2.0
    
    x1_p = Y_pred[:, :, :, 1:2] - pred_x_w / 2.0
    x2_p = Y_pred[:, :, :, 1:2] + pred_x_w / 2.0
    
    y1_p = Y_pred[:, :, :, 3:4] - pred_x_h / 2.0
    y2_p = Y_pred[:, :, :, 3:4] + pred_x_h / 2.0
    
    cond1 = x2_t < x1_p
    cond2 = x2_p < x1_t
    cond3 = y2_t < y1_p
    cond4 = y2_p < y1_t
    cond_all = tf.logical_or(tf.logical_or(tf.logical_or(cond1, cond2), cond3), cond4)
    
    ious_np = np.zeros([Y_pred.shape[0], Y_pred.shape[1], Y_pred.shape[2], 1], dtype=np.float32)
    
    far_x = tf.where(x2_t < x2_p, x2_t, x2_p)
    near_x = tf.where(x1_t > x1_p, x1_t, x1_p)
    far_y = tf.where(y2_t < y2_p, y2_t, y2_p)
    near_y = tf.where(y1_t > y1_p, y1_t, y1_p)
    
    inter_area = (far_x - near_x + 1.0) * (far_y - near_y + 1.0)
    true_box_area = (x2_t - x1_t + 1.0) * (y2_t - y1_t + 1.0)
    pred_box_area = (x2_p - x1_p + 1.0) * (y2_p - y1_p + 1.0)
    iou = inter_area / (true_box_area + pred_box_area - inter_area)
    iou = tf.where(cond_all, ious_np, iou)
    return iou

In [ ]:
w_val = 0.1
W_M_1 = tf.Variable(np.random.uniform(-w_val, w_val, [3, 3, 3, 32]), dtype=tf.float32, name='WM1')
W_M_1_1 = tf.Variable(np.random.uniform(-w_val, w_val, [3, 3, 32, 64]), dtype=tf.float32, name='WM11')
W_M_2 = tf.Variable(np.random.uniform(-w_val, w_val, [3, 3, 64, 128]), dtype=tf.float32, name='WM2')
W_M_2_1 = tf.Variable(np.random.uniform(-w_val, w_val, [3, 3, 128, 256]), dtype=tf.float32, name='WM21')
W_M_2_2 = tf.Variable(np.random.uniform(-w_val, w_val, [3, 3, 256, 256]), dtype=tf.float32, name='WM22')
W_M_2_3 = tf.Variable(np.random.uniform(-w_val, w_val, [3, 3, 256, 512]), dtype=tf.float32, name='WM23')
W_M_2_4 = tf.Variable(np.random.uniform(-w_val, w_val, [2, 2, 512, 256]), dtype=tf.float32, name='WM24')
W_M_3 = tf.Variable(np.random.uniform(-w_val, w_val, [2*2*256, 256]), dtype=tf.float32, name='WM3')
W_M_3_1 = tf.Variable(np.random.uniform(-w_val, w_val, [256, 128]), dtype=tf.float32, name='WM31')

W_M_C = tf.Variable(np.random.uniform(-w_val, w_val, [128*5, 128]), dtype=tf.float32, name='WMC')

W_M_4_F = tf.Variable(np.random.uniform(-w_val, w_val, [128, len(loader.category_dict)*5]), 
                      dtype=tf.float32, name='WM4F')

params = [W_M_1, W_M_1_1, W_M_2, W_M_2_1, W_M_2_2, W_M_2_3, 
          W_M_2_4, W_M_3, W_M_3_1, W_M_4_F, W_M_C]

In [ ]:
def feed_forward(X):
    X_flat = tf.reshape(X, [X.shape[0]*X.shape[1], X.shape[2], X.shape[3], X.shape[4]])
    encode_conv = tf.nn.leaky_relu(tf.nn.conv2d(X_flat, W_M_1, [1, 2, 2, 1], 'SAME'), alpha=0.1)
    encode_conv = tf.nn.leaky_relu(tf.nn.conv2d(encode_conv, W_M_1_1, [1, 2, 2, 1], 'SAME'), alpha=0.1)
    share_conv = encode_conv

    share_conv = tf.nn.leaky_relu(tf.nn.conv2d(share_conv, W_M_2, [1, 2, 2, 1], 'SAME'), alpha=0.1)
    share_conv = tf.nn.leaky_relu(tf.nn.conv2d(share_conv, W_M_2_1, [1, 2, 2, 1], 'SAME'), alpha=0.1)
    share_conv = tf.nn.leaky_relu(tf.nn.conv2d(share_conv, W_M_2_2, [1, 2, 2, 1], 'SAME'), alpha=0.1)
    share_conv = tf.nn.leaky_relu(tf.nn.conv2d(share_conv, W_M_2_3, [1, 1, 1, 1], 'VALID'), alpha=0.1)
    share_conv = tf.nn.leaky_relu(tf.nn.conv2d(share_conv, W_M_2_4, [1, 1, 1, 1], 'VALID'), alpha=0.1)
    #print(share_conv.shape)
    
    share_flat = tf.reshape(share_conv, [share_conv.shape[0], W_M_3.shape[0]])
    flat_1 = tf.nn.leaky_relu(tf.matmul(share_flat, W_M_3), alpha=0.1)
    flat_2 = tf.nn.leaky_relu(tf.matmul(flat_1, W_M_3_1), alpha=0.1)
        
    flat_2 = tf.reshape(flat_2, [X.shape[0], X.shape[1], flat_2.shape[1]])
    padded = tf.pad(flat_2, [[1, 1], [1, 1], [0, 0]])
    indices = index_lookup_generator(X.shape[0], X.shape[1]) + 1
    collaborations = tf.gather_nd(padded, indices)
    collaborations = tf.reshape(collaborations, [X.shape[0], X.shape[1], collaborations.shape[-1]*5])
    collaborations = tf.reshape(collaborations, [X.shape[0] * X.shape[1], collaborations.shape[-1]])
    final =  tf.nn.leaky_relu(tf.matmul(collaborations, W_M_C), alpha=0.1)
    
    for i in range(2):
        final = tf.reshape(final, [X.shape[0], X.shape[1], final.shape[1]])
        padded = tf.pad(final, [[1, 1], [1, 1], [0, 0]])
        collaborations = tf.gather_nd(padded, indices)
        collaborations = tf.reshape(collaborations, [X.shape[0], X.shape[1], collaborations.shape[-1]*5])
        collaborations = tf.reshape(collaborations, [X.shape[0] * X.shape[1], collaborations.shape[-1]])
        final =  tf.nn.leaky_relu(tf.matmul(collaborations, W_M_C), alpha=0.1)
    
    Y_pred = tf.matmul(final, W_M_4_F)
    Y_pred = tf.reshape(Y_pred, [Y_pred.shape[0], len(loader.category_dict), 5])
    Y_pred = tf.reshape(Y_pred, Y.shape)
    
    class_pred = tf.nn.sigmoid(Y_pred[:, :, :, 0])
    xc_pred = Y_pred[:, :, :, 1]
    yc_pred = Y_pred[:, :, :, 3]
    xw_pred = tf.square(Y_pred[:, :, :, 2])
    yw_pred = tf.square(Y_pred[:, :, :, 4])
    
    #Y_pred = tf.nn.leaky_relu(Y_pred, alpha=0.5)
    Y_pred = tf.stack([class_pred, 
                       xc_pred, 
                       xw_pred, 
                       yc_pred, 
                       yw_pred], axis=3)

    return Y_pred

In [ ]:
# 1 must be added for shifted
def index_lookup_generator(w, h):
    indices = np.zeros([w, h, 5, 2], dtype=np.int32)
    for i in range(w):
        for j in range(h):
            l = [[i, j-1], [i, j+1], [i, j], [i-1, j], [i+1, j]]
            indices[i, j] = l
    return indices

In [ ]:
def save_weights(params, extention=''):
    for p in params:
        np.save("weights/"+p.name.replace(':', '_')+'_'+extention, p.numpy())
        
def load_weights(params, extention=''):
    for p in params:
        p.assign(np.load("weights/"+p.name.replace(':', '_')+'_'+extention+'.npy'))

In [ ]:
load_weights(params, 'bckp')

In [ ]:
Y_preds = []
Ys = []

stride = 50
patch_size = 150

for i in range(loader.image_count):
    X, Y, image, cmap = loader.gather(i, stride, patch_size, 0.1)
    if X is not None:
        Y_pred = feed_forward(X)
        iou_calculations.append(calc_iou(Y_pred, Y).numpy())
        Y_preds.append(Y_pred.numpy())
        Ys.append(Y)
        print(i)

In [ ]:
Yp = np.asarray(Y_preds)
Yt = np.asarray(Ys)

print(Yp.shape)
print(Yt.shape)

np.save('Yp', Yp)
np.save('Yt', Yt)

In [2]:
Yp = np.load('Yp.npy')
Yt = np.load('Yt.npy')

In [16]:
iou_calculations = []
for i in range(Yp.shape[0]):
    iou_calculations.append(calc_iou(Yp[i], Yt[i]).numpy())
ious = np.asarray(iou_calculations)
np.save('ious', ious)

In [7]:
print(Yp.shape)
print(Yt.shape)
print(ious.shape)

(9748, 15, 26, 10, 5)
(9748, 15, 26, 10, 5)
(9748, 15, 26, 10, 1)


In [18]:
1-np.mean(ious)

0.010187387466430664

In [32]:
Ypa = np.asarray(Yp[:, :, :, :, 0], dtype=np.float32)
Yta = np.asarray(Yt[:, :, :, :, 0], dtype=np.float32)
print(Ypa.shape)
print(ious.shape)

(9748, 15, 26, 10)
(9748, 15, 26, 10, 1)


In [33]:
rv = 1-ious
rv = np.reshape(ious, [ious.shape[0], ious.shape[1], ious.shape[2], ious.shape[3]])
predictions = Ypa[rv>0.5] > 0.5
trues = Yta[rv>0.5] > 0.5


0.9898126
(37942081,)
(38017200,)
0.001975921425039193


In [9]:
np.save('Ypa' ,np.asarray(Yp[:, :, :, :, 0], dtype=np.float32))
np.save('Yta' ,np.asarray(Yt[:, :, :, :, 0], dtype=np.float32))

In [14]:
ious[10, 10, 10]

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]], dtype=float32)